In [9]:
import tensorflow as tf

from tensorflow.keras import datasets, models, layers

import matplotlib.pyplot as plt

def reshaper(X):
    X = X.reshape(list(X.shape) + [1])
    return X

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

first_model = models.Sequential([
    layers.Conv2D(14, (3, 3), padding="same", activation='relu', input_shape=(28, 28, 1), kernel_initializer="he_normal"),
    layers.MaxPooling2D(),
    layers.Conv2D(28, (3, 3), padding="same", activation='relu', kernel_initializer="he_normal"),
    layers.MaxPooling2D(),
    layers.Conv2D(56, (3, 3), padding="same", activation='relu', kernel_initializer="he_normal"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu', kernel_initializer="he_normal"),
    layers.Dense(10, activation='softmax', kernel_initializer="he_normal"),
])

train_images = reshaper(train_images)
test_images = reshaper(test_images)
test_labels = reshaper(test_labels)
train_labels = reshaper(train_labels)

first_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 14)        140       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 14)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 28)        3556      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 28)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 7, 56)          14168     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 3, 3, 56)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 504)              

In [ ]:

first_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = first_model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

Epoch 1/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.1590 - accuracy: 0.9506 - val_loss: 0.0534 - val_accuracy: 0.9839
Epoch 2/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0487 - accuracy: 0.9851 - val_loss: 0.0487 - val_accuracy: 0.9847
Epoch 3/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0369 - accuracy: 0.9885 - val_loss: 0.0400 - val_accuracy: 0.9876
Epoch 4/10
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0285 - accuracy: 0.9908 - val_loss: 0.0279 - val_accuracy: 0.9902
Epoch 5/10
 957/1875 [==============>...............] - ETA: 12s - loss: 0.0200 - accuracy: 0.9936

In [ ]:
model = models.Sequential([
    layers.Conv2D(14, (3, 3), padding="same", activation='relu', input_shape=(28, 28, 1), kernel_initializer="he_normal"),
    layers.MaxPooling2D(),
    layers.Conv2D(28, (3, 3), padding="same", activation='relu', kernel_initializer="he_normal"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(32, activation='relu', kernel_initializer="he_normal"),
    layers.Dense(10, activation='softmax', kernel_initializer="he_normal"),
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:

CM = [[0 for _ in range(10)] for _ in range(10)]
ims = [[(None, None) for _ in range(10)] for _ in range(10)]
pred_prob = model.predict(test_images)
pred = model.predict_classes(test_images)
for i in range(10000):
    CM[test_labels[i][0]][pred[i]] += 1
    
    (cur, _) = ims[test_labels[i][0]][pred[i]]
    if cur is None or cur < pred_prob[i][pred[i]]:
        ims[test_labels[i][0]][pred[i]] = (pred_prob[i][pred[i]], i)
for string in CM:
    print(string)

In [ ]:
plt.figure(figsize=(20,20))

for i in range(10):
    for j in range(10):
        plt.subplot(10,10, 10 * i + j + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        (_, k) = ims[i][j]
        if not k is None:
            plt.imshow(test_images[k], cmap=plt.cm.binary)
            plt.xlabel("Real {}, Pred {}".format(i, j))